# Source localization approximation

This example illustrates source localization via multi-dimensional scaling
using the Julia language.

This entire page was generated using a single Julia file:
[source-local.jl](https://github.com/JeffFessler/book-mmaj-demo/blob/main/docs/lit/demos/06/source-local.jl).

First we add the Julia packages that are need for this demo.
Change `false` to `true` in the following code block
if you are using any of the following packages for the first time.

In [ ]:
if false
    import Pkg
    Pkg.add([
        "LinearAlgebra"
        "MIRTjim"
        "Plots"
        "Random"
        "Statistics"
        "InteractiveUtils"
    ])
end

Now tell this Julia session to use the following packages for this example.
Run `Pkg.add()` in the preceding code block first, if needed.

In [ ]:
using LinearAlgebra: svd, norm, Diagonal
using Statistics: mean
using Random: seed!
using Plots; default(label="", markerstrokecolor=:auto)
using MIRTjim: jim, prompt
using InteractiveUtils: versioninfo

The following line is helpful when running this jl-file as a script;
this way it will prompt user to hit a key after each image is displayed.

In [ ]:
isinteractive() && prompt(:prompt);
isinteractive() && jim(:prompt);

## Generate data

Coordinates - can you identify the pattern?  probably not...

In [ ]:
C0 = [
3 2.5 2 2 2 2 2 1 0 0 1 1 1 1 0 0 1 2 2.5 3  4 4.5 5 6 7 7 6 6 6 6 7 7 6 5 5 5 5 5 4.5 4;
2 3.0 4 3 2 1 0 0 0 1 1 2 3 4 4 5 5 5 4.0 3  3 4.0 5 5 5 4 4 3 2 1 1 0 0 0 1 2 3 4 3.0 2
];

Interpolate the locations to make the final set more familiar

In [ ]:
C2 = (C0[:,2:end] + C0[:,1:end-1])/2
Ce = (C0[:,1] + C0[:,end])/2
C3 = [C2 Ce]
C4 = [C0; C3]
C = reshape(C4, 2, :);

### Compute J × J distance array and display it

In [ ]:
J = size(C,2) # number of points
D = [norm(C[:,j] - C[:,i]) for i=1:J, j=1:J] # "comprehension" in julia!
jim(D, "D", color=:cividis)

### MDS algorithm

Compute Gram matrix by de-meaning squared distance matrix

In [ ]:
S = D.^2 # squared distances
G = S .- mean(S,dims=1) # trick: use "broadcasting" feature of julia
G = G .- mean(G,dims=2) # now we have de-meaned the columns and rows of S
G = -1/2 * G
jim(G, "G", color=:cividis) # still cannot determine visually the point locations

Examine singular values

In [ ]:
(_, σ, V) = svd(G) # svd returns singular values in descending order
scatter(σ, label="singular values") # two nonzero (d=2)

In [ ]:
prompt()

### Estimate the source locations using rank=2

In [ ]:
Ch = Diagonal(sqrt.(σ[1:2])) * V[:,1:2]' # here is the key step

### Plot estimated source locations

In [ ]:
scatter(Ch[1,:], -Ch[2,:], xtick=-4:4, ytick=-3:3, label="", aspect_ratio=1,
 title="Location estimates")

In [ ]:
prompt()

## Noisy case

In [ ]:
seed!(0)
Dn = D + 0.3 * randn(size(D))
Sn = Dn.^2
Gn = Sn .- mean(Sn,dims=1)
Gn = Gn .- mean(Gn,dims=2) # de-meaned
Gn = -1/2 * Gn
jim(Gn, "G noisy", color=:cividis) # still cannot determine visually the point locations

Singular values

In [ ]:
(_, sn, Vn) = svd(Gn)
scatter(abs.(sn), label="singular values") # two >> 0

In [ ]:
prompt()

### Plot estimated source locations from noisy distance measurements

In [ ]:
Cn = Diagonal(sqrt.(sn[1:2])) * Vn[:,1:2]' # here is the key step
scatter(Cn[1,:], -Cn[2,:], xtick=-4:4, ytick=-3:3, label="", aspect_ratio=1,
 title="Location estimates")

In [ ]:
prompt()

## Equilateral triangle example

In [ ]:
G = [-2 1 1; 1 -2 1; 1 1 -2] / (-6.)
(~, σ, V) = svd(G)
Ch = Diagonal(sqrt.(σ[1:2])) * V[:,1:2]'
scatter(Ch[1,:], Ch[2,:], aspect_ratio=1, label="",
 title="Location estimates")

In [ ]:
prompt()

## Reproducibility

This page was generated with the following version of Julia:

In [ ]:
io = IOBuffer(); versioninfo(io); split(String(take!(io)), '\n')

And with the following package versions

In [ ]:
import Pkg; Pkg.status()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*